In [1]:
import os
import pickle as pkl
import numpy as np

import torch
from torch import nn
import torch.nn.functional as F
from torchvision import transforms
from torchvision.datasets import MNIST
from torch.utils.data import Dataset, DataLoader, random_split
import pytorch_lightning as pl

from nuscenes import NuScenes
from nuscenes.eval.prediction.splits import get_prediction_challenge_split
from nuscenes.prediction import PredictHelper

from nuscenes.prediction.models.backbone import ResNetBackbone
from nuscenes.prediction.models.mtp import MTP
from nuscenes.prediction.models.covernet import CoverNet, ConstantLatticeLoss

import matplotlib.pyplot as plt
%matplotlib inline

from nuscenes.prediction.input_representation.static_layers import StaticLayerRasterizer
from nuscenes.prediction.input_representation.agents import AgentBoxesWithFadedHistory
from nuscenes.prediction.input_representation.interface import InputRepresentation
from nuscenes.prediction.input_representation.combinators import Rasterizer

In [8]:
class NuScenesDataset(Dataset):
    
    def __init__(self, dataroot, nuscenes, mode='train', seconds_of_history=1):
        self.dataroot = dataroot
        self.seconds_of_history = seconds_of_history
        self.nuscenes = nuscenes
            
        self.helper = PredictHelper(self.nuscenes)
        self.static_layer_rasterizer = StaticLayerRasterizer(self.helper)
        self.agent_rasterizer = AgentBoxesWithFadedHistory(self.helper, seconds_of_history=self.seconds_of_history)
        self.input_representation = InputRepresentation(self.static_layer_rasterizer, self.agent_rasterizer, Rasterizer())
        
        if mode == 'train':
            # We're in test stage then
            self.predictions = get_prediction_challenge_split("train", dataroot=self.dataroot)
        elif mode == 'val':
            self.predictions = get_prediction_challenge_split("val", dataroot=self.dataroot)
        else:
            raise ValueError('Incorrect mode was provided. mode can be one of the following values: {train, val}')
                    
    def __len__(self):
        return len(self.predictions)
    
    def __getitem__(self, idx):
        # Get and load image
        instance_token, sample_token = self.predictions[idx].split("_")
        anns = [ann for ann in self.nuscenes.sample_annotation if ann['instance_token'] == instance_token]
        img = self.input_representation.make_input_representation(instance_token, sample_token)
        img = torch.Tensor(img).permute(2, 0, 1).unsqueeze(0)
      
        agent_state_vector = torch.Tensor([[self.helper.get_velocity_for_agent(instance_token, sample_token),
                                    self.helper.get_acceleration_for_agent(instance_token, sample_token),
                                    self.helper.get_heading_change_rate_for_agent(instance_token, sample_token)]])

        agent_state_vector[torch.isnan(agent_state_vector)] = -1
        label = self.helper.get_future_for_agent(instance_token, sample_token, 6, in_agent_frame=True)
        label = np.expand_dims(label, 0)
        
        return img, agent_state_vector, label

In [14]:
class NuScenesDataModule(pl.LightningDataModule):

      def __init__(self, dataroot, nuscenes, batch_size=32, seconds_of_history=1):
          super().__init__()
          self.batch_size = batch_size
          self.dataroot = dataroot
          self.nuscenes = nuscenes
          self.seconds_of_history = seconds_of_history
            
#           self.nuscenes = NuScenes('v1.0-trainval', dataroot=self.dataroot)
#           self.helper = PredictHelper(self.nuscenes)
#           self.static_layer_rasterizer = StaticLayerRasterizer(helper)
#           self.agent_rasterizer = AgentBoxesWithFadedHistory(self.helper, seconds_of_history=self.seconds_of_history)
#           self.input_representation = InputRepresentation(self.static_layer_rasterizer, self.agent_rasterizer, Rasterizer())

      # OPTIONAL, called for every GPU/machine (assigning state is OK)
      def setup(self, stage = None):
          
          # split dataset
          if stage == 'fit':
                 train = NuScenesDataset(self.dataroot, nuscenes, mode='train', seconds_of_history=self.seconds_of_history) 
                 self.train, self.val = random_split(train, [int(0.9*len(train)), len(train) - int(0.9*len(train))])
                 print(len(self.train), len(self.val), len(train))
          if stage == 'test':
                 self.test = NuScenesDataset(self.dataroot, nuscenes, mode='val', seconds_of_history=self.seconds_of_history) 

      # return the dataloader for each split
      def train_dataloader(self):
          train = DataLoader(self.train, batch_size=self.batch_size, num_workers=16)
          return train

      def val_dataloader(self):
          val = DataLoader(self.val, batch_size=len(self.val), num_workers=16)
          return val

      def test_dataloader(self):
          test = DataLoader(self.test, batch_size=len(self.test), num_workers=16)
          return test

In [10]:
class PredictionModel(pl.LightningModule):

    def __init__(self, num_modes=64):
        super().__init__()
        self.num_modes = num_modes
        backbone = ResNetBackbone('resnet50')
        
        # Note that the value of num_modes depends on the size of the lattice used for CoverNet.
        self.covernet = CoverNet(backbone, num_modes=self.num_modes)
        
        # Constant trajectory set
        PATH_TO_EPSILON_8_SET = "/home/mitkina/git-repos/Waymo/data/sets/nuscenes/covernet_traj_set/nuscenes-prediction-challenge-trajectory-sets/epsilon_8.pkl"
        trajectories = pkl.load(open(PATH_TO_EPSILON_8_SET, 'rb'))
        
        # Saved them as a list of lists
        trajectories = torch.Tensor(trajectories)
        self.loss = ConstantLatticeLoss(trajectories)
#         print("trajectories: ", trajectories.shape)

        # mtp = MTP(backbone, num_modes=2)
        
        # How to pass the nuscenes data helper to the model module?
#         self.helper = PredictHelper(nuscenes)
        
        # Image representation
#         static_layer_rasterizer = StaticLayerRasterizer(helper)
#         agent_rasterizer = AgentBoxesWithFadedHistory(helper, seconds_of_history=1)
#         self.mtp_input_representation = InputRepresentation(static_layer_rasterizer, agent_rasterizer, Rasterizer())

    def forward(self, x):
        # in lightning, forward defines the prediction/inference actions
        logits = self.covernet(x)
        return logits

    def training_step(self, batch, batch_idx):
        # training_step defined the train loop.
        # It is independent of forward
        image_tensor, agent_state_vector, labels = batch
        agent_state_vector = torch.squeeze(agent_state_vector, 1)
        image_tensor = torch.squeeze(image_tensor, 1)
#         print("data", image_tensor.shape, agent_state_vector.shape, labels.shape)
        
        logits = self.covernet(image_tensor, agent_state_vector)
#         print("logits", logits.shape)
        loss = self.loss(logits, labels)
        # Logging to TensorBoard by default
        self.log('train_loss', loss)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

In [16]:
dataroot = '/home/mitkina/git-repos/Waymo/data/sets/nuscenes'
nuscenes = NuScenes('v1.0-trainval', dataroot=dataroot)
dataset = NuScenesDataModule(dataroot, nuscenes, batch_size=16)

Loading NuScenes tables for version v1.0-trainval...
23 category,
8 attribute,
4 visibility,
64386 instance,
12 sensor,
10200 calibrated_sensor,
2631083 ego_pose,
68 log,
850 scene,
34149 sample,
2631083 sample_data,
1166187 sample_annotation,
4 map,
Done loading in 22.718 seconds.
Reverse indexing ...
Done reverse indexing in 5.6 seconds.


In [17]:
dataset.setup(stage='fit')
dataset.setup(stage='test')
len(dataset.test)

28967 3219 32186


9041

In [ ]:
# init model
model = PredictionModel()

# most basic trainer, uses good defaults (auto-tensorboard, checkpoints, logs, and more)
# trainer = pl.Trainer(gpus=8) (if you have GPUs)
trainer = pl.Trainer(gpus=1)
trainer.fit(model, dataset)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type     | Params
--------------------------------------
0 | covernet | CoverNet | 32.2 M
--------------------------------------
32.2 M    Trainable params
0         Non-trainable params
32.2 M    Total params
128.701   Total estimated model params size (MB)


trajectories:  torch.Size([64, 12, 2])


Training: 0it [00:00, ?it/s]

data torch.Size([16, 3, 500, 500]) torch.Size([16, 3]) torch.Size([16, 1, 12, 2])
logits torch.Size([16, 64])
data torch.Size([16, 3, 500, 500]) torch.Size([16, 3]) torch.Size([16, 1, 12, 2])
logits torch.Size([16, 64])
data torch.Size([16, 3, 500, 500]) torch.Size([16, 3]) torch.Size([16, 1, 12, 2])
logits torch.Size([16, 64])
data torch.Size([16, 3, 500, 500]) torch.Size([16, 3]) torch.Size([16, 1, 12, 2])
logits torch.Size([16, 64])
data torch.Size([16, 3, 500, 500]) torch.Size([16, 3]) torch.Size([16, 1, 12, 2])
logits torch.Size([16, 64])
data torch.Size([16, 3, 500, 500]) torch.Size([16, 3]) torch.Size([16, 1, 12, 2])
logits torch.Size([16, 64])
data torch.Size([16, 3, 500, 500]) torch.Size([16, 3]) torch.Size([16, 1, 12, 2])
logits torch.Size([16, 64])
data torch.Size([16, 3, 500, 500]) torch.Size([16, 3]) torch.Size([16, 1, 12, 2])
logits torch.Size([16, 64])
data torch.Size([16, 3, 500, 500]) torch.Size([16, 3]) torch.Size([16, 1, 12, 2])
logits torch.Size([16, 64])
data torch

In [ ]:
# test
trainer.test(datamodule=dm)

In [ ]:
model = LitModel.load_from_checkpoint(path)

# # load the ckpt
# ckpt = torch.load('path/to/checkpoint.ckpt')

# # equivalent to the above
# model = LitModel()
# model.load_state_dict(ckpt['state_dict'])